<a href="https://colab.research.google.com/github/balawillgetyou/dy/blob/master/LSTMGloVeTextClassifier20190927.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import codecs, os, numpy as np, pandas as pd
pd.set_option('display.max_colwidth', -1) #to prevent cell display truncation

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers 
#initializers, regularizers, constraints are not explicitly called

from sklearn.utils import shuffle
from sklearn import preprocessing  
from sklearn.model_selection import train_test_split

from keras.callbacks import EarlyStopping, ModelCheckpoint#options for when to stop training 
#and to save only the best performing iteration 

import re#for text preprocessing (checking for non words)

#metrics calculation
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_recall_fscore_support as score

Using TensorFlow backend.


In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
temp_1 = open("/content/gdrive/My Drive/Colab Notebooks/WhoseLineIsItAnywayTRAIN.csv", 'r', encoding='latin-1') 
WhoseLineData = pd.read_csv(temp_1)
print(WhoseLineData.shape)
#temp_1 = open("/content/gdrive/My Drive/Colab Notebooks/WhoseLineIsItAnywayTEST.csv", 'r', encoding='latin-1') 
#WhoseLineDataTest = pd.read_csv(temp_1)
#print(WhoseLineDataTest.shape)

(18977, 2)


In [0]:
#knock out the many white spaces & non words 
WhoseLineData['text'].replace(r'\s+', ' ', regex=True, inplace=True)
#WhoseLineDataTest['text'].replace(r'\s+', ' ', regex=True, inplace=True)

WhoseLineData['text'] = WhoseLineData['text'].str.lower().str.replace("[^a-zA-Z]", " ")
#WhoseLineDataTest['text'] = WhoseLineDataTest['text'].str.lower().str.replace("[^a-zA-Z]", " ")

In [6]:
#unique word count, needed to set the max_features
uniqueWords = set()
WhoseLineData['text'].str.split().apply(uniqueWords.update)
print('# of unique words',len(uniqueWords))
print('Sample of unique words',list(uniqueWords)[1070:1100])

# of unique words 65924
Sample of unique words ['hatfield', 'pebbles', 'brazils', 'furthermore', 'apostle', 'teats', 'avow', 'siegfried', 'origane', 'snowy', 'suspender', 'deny', 'particularized', 'reszkes', 'roderick', 'junkum', 'fedosie', 'sage', 'perturb', 'erfurt', 'couldn', 'loom', 'squanderings', 'glad', 'flighty', 'villains', 'kindling', 'mistress', 'assassin', 'jesters']


In [0]:
#to get an idea for the typical length of each text, to set the maxlen parameter later
WhoseLineData['textLength'] = WhoseLineData['text'].str.split().str.len()

In [8]:
WhoseLineData.describe()

,author,textLength
count,18977.000000,18977.000000
mean,3.667176,400.118617
std,2.678708,312.926616
min,0.000000,23.000000
25%,2.000000,251.000000
50%,4.000000,339.000000
75%,5.000000,465.000000
max,9.000000,5908.000000


In [9]:
#this is needed to define the number of nodes in the final layer of the neural network
WhoseLineData['author'].unique()

array([2, 0, 5, 4, 8, 3, 9, 6, 1, 7])

In [10]:
#finding the shortest text, displaying the row + splitting & unique words count (set)
print('Row with the shortest text is:', WhoseLineData['textLength'].idxmin())
print('Shortest text:', '\n',WhoseLineData['text'][135])
print('List of words:', '\n', WhoseLineData['text'][135].split())
print('#of words:', len(WhoseLineData['text'][135].split()))
print('#of unique words:', len(set(WhoseLineData['text'][135].split())))

Row with the shortest text is: 135
Shortest text: 
 but what if my heart had failed me  or i had shrunk from making up my mind      no  tomorrow all shall be ended  
List of words: 
 ['but', 'what', 'if', 'my', 'heart', 'had', 'failed', 'me', 'or', 'i', 'had', 'shrunk', 'from', 'making', 'up', 'my', 'mind', 'no', 'tomorrow', 'all', 'shall', 'be', 'ended']
#of words: 23
#of unique words: 21


In [11]:
#finding the longest text, splitting & unique words count (set), and display some of the
#unique words. Note the presence of special characters despite running regex above
print('Row with the longest text is:', WhoseLineData['textLength'].idxmax())
print('#of unique words:', len(set(WhoseLineData['text'][16960].split())))
print('Sample of unique words:', '\n', list(set(WhoseLineData['text'][16960].split()))[0:50])

Row with the longest text is: 16960
#of unique words: 1110
Sample of unique words: 
 ['been', 'souls', 'comes', 'daughter', 'back', 'steep', 'shadow', 'damned', 'sat', 'orders', 'visible', 'seen', 'break', 'palaces', 'turn', 'loving', 'wooing', 'particle', 'told', 'elderly', 'an', 'second', 'night', 'coined', 'smitten', 'answer', 'none', 'orchard', 'belly', 'lose', 'to', 'condoled', 'women', 'this', 'read', 'shop', 'steamers', 'often', 'walk', 'bird', 'thinking', 'fortunate', 'rubbing', 'target', 'pour', 'sorrow', 'ava', 'orchards', 'lies', 'explain']


In [0]:
EMBEDDING_FILE='/content/gdrive/My Drive/Colab Notebooks/glove.6B.50d.txt'#word emdedding file
embed_size = 50 # word vector/ embedding size
max_features = 20000 # number unique words to use (i.e num rows in embedding vector)
maxlen = 250 # max number of words used, from each row of text

In [13]:
#splitting train and test data
train, test = train_test_split(WhoseLineData, test_size=0.15)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

#for final submission
#train = WhoseLineData
#test = WhoseLineDataTest

list_sentences_train = train["text"]
list_sentences_test = test["text"]

#one hot encoding the labels
y = train['author']
y=y.values.reshape(-1,1)
enc = preprocessing.OneHotEncoder(categorical_features = [0])
y1 = enc.fit_transform(y).toarray()

y = test['author']
y=y.values.reshape(-1,1)
y_test = enc.fit_transform(y).toarray()#reuse the same encoder that transformed training data

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data w

In [14]:
len(list_sentences_test)#confirming test data size

2847

In [0]:
#convert texts to lists of word indices + padding lists so they are all of equal length
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [0]:
#loading word embedding
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

In [17]:
#this and the next cell convert word indices to word embeddings, where available. use random initialization for others.
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.


(0.020940498, 0.6441043)

In [0]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index)+1)
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [0]:
#definining a bi-directional LSTM
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.05, recurrent_dropout=0.05))(x)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.05, recurrent_dropout=0.05))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(10, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
#training for more epochs seemed to help. the callback helps prevents over doing it. probably a good idea to add 
#checkpoint as well to save the best performing iteration. checkpoint = [ModelCheckpoint(filepath='models.hdf5')]
 
callbacks = [EarlyStopping(monitor='val_loss', patience=2)]

In [32]:
model.fit(X_t, y1, batch_size=1000, epochs=30, callbacks=callbacks, validation_split=0.1)

Train on 14517 samples, validate on 1613 samples
Epoch 1/30
14517/14517 [==============================] - 47s 3ms/step - loss: 2.2397 - acc: 0.1601 - val_loss: 2.1598 - val_acc: 0.1878
Epoch 2/30
14517/14517 [==============================] - 42s 3ms/step - loss: 2.1297 - acc: 0.1938 - val_loss: 2.0613 - val_acc: 0.1897
Epoch 3/30
14517/14517 [==============================] - 42s 3ms/step - loss: 2.0417 - acc: 0.2272 - val_loss: 1.9398 - val_acc: 0.2771
Epoch 4/30
14517/14517 [==============================] - 42s 3ms/step - loss: 1.9324 - acc: 0.2576 - val_loss: 1.8307 - val_acc: 0.3007
Epoch 5/30
14517/14517 [==============================] - 42s 3ms/step - loss: 1.8246 - acc: 0.2949 - val_loss: 1.7283 - val_acc: 0.3236
Epoch 6/30
14517/14517 [==============================] - 41s 3ms/step - loss: 1.7110 - acc: 0.3455 - val_loss: 1.5486 - val_acc: 0.3962
Epoch 7/30
14517/14517 [==============================] - 42s 3ms/step - loss: 1.6016 - acc: 0.3755 - val_loss: 1.4603 - val_acc:

In [33]:
y_pred = model.predict([X_te], batch_size=1000, verbose=1)

2847/2847 [==============================] - 4s 1ms/step


In [34]:
y_test = np.argmax(y_test, axis=1)
y_pred = np.argmax(y_pred, axis=1)
precision, recall, fscore, support = score(y_test, y_pred, labels = [0,1,2,3,4,5,6,7,8,9])
results1 = pd.concat((pd.Series([0,1,2,3,4,5,6,7,8,9]), pd.Series(precision), pd.Series(recall), pd.Series(fscore), pd.Series(support)), axis=1, join = 'outer')
results1.columns=('label','precision', 'recall', 'fscore', 'support')
print('Results','\n','*'*50,'\n',round(results1,2))

Results 
 ************************************************** 
    label  precision  recall  fscore  support
0  0      0.94       0.91    0.93    598    
1  1      0.78       0.80    0.79    99     
2  2      0.93       0.93    0.93    395    
3  3      0.96       0.96    0.96    196    
4  4      0.92       0.93    0.92    492    
5  5      0.83       0.89    0.86    482    
6  6      0.87       0.86    0.87    118    
7  7      0.78       0.73    0.75    162    
8  8      0.92       0.73    0.81    151    
9  9      0.75       0.86    0.80    154    
